<a href="https://colab.research.google.com/github/PurpleUniverse/ModelArchitectures/blob/main/ML_ModelArchitectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount the Google Drive and load the EMNIST dataset**

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import drive
from sklearn.model_selection import train_test_split

# Mount to Google Drive for access of data file
drive.mount('/content/drive')

pathTrain = "/content/drive/My Drive/EMNIST/emnist-balanced-train.csv"
train = pd.read_csv(pathTrain).to_numpy()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Prepare the data**

In [5]:
# Separate labels and features
X = train[:, 1:].astype('float32') / 255.0  # Normalize pixel values
y = train[:, 0].astype('int')

# Reshape the data for CNN (28, 28, 1)
X = X.reshape(-1, 28, 28, 1)

# Step 2: Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

**Build and Train the CNN Model**

In [12]:
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout layer
    model.add(layers.Dense(47, activation='softmax'))  # 47 classes for EMNIST Balanced
    return model

# Train the CNN Model
cnn_model = create_cnn_model()
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Fit the model
cnn_model.fit(X_train, y_train,
               epochs=20,
               batch_size=64,
               validation_data=(X_val, y_val),
               callbacks=[early_stopping])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.3867 - loss: 2.2127 - val_accuracy: 0.8072 - val_loss: 0.5662
Epoch 2/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7319 - loss: 0.8405 - val_accuracy: 0.8386 - val_loss: 0.4576
Epoch 3/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7793 - loss: 0.6732 - val_accuracy: 0.8517 - val_loss: 0.4078
Epoch 4/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8050 - loss: 0.5935 - val_accuracy: 0.8574 - val_loss: 0.3860
Epoch 5/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8166 - loss: 0.5509 - val_accuracy: 0.8623 - val_loss: 0.3772
Epoch 6/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8277 - loss: 0.5155 - val_accuracy: 0.8645 - val_loss: 0.3687
Epoch 7/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8327 - loss: 0.4860 - val_accuracy: 0.8687 - val_loss: 0.3587
Epoch 8/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8395 - loss: 0.4677 -

**Build and Train the MLP Model**

In [8]:
def create_mlp_model():
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(784,)))  # Flatten the input
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout layer
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout layer
    model.add(layers.Dense(47, activation='softmax'))  # 47 classes for EMNIST Balanced
    return model

# Train the MLP Model
mlp_model = create_mlp_model()
mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping_mlp = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Fit the model
mlp_model.fit(X_train.reshape(-1, 784), y_train,
               epochs=20,
               batch_size=64,
               validation_data=(X_val.reshape(-1, 784), y_val),
               callbacks=[early_stopping_mlp])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.2323 - loss: 2.8597 - val_accuracy: 0.6573 - val_loss: 1.2092
Epoch 2/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4967 - loss: 1.7108 - val_accuracy: 0.7007 - val_loss: 1.0257
Epoch 3/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5418 - loss: 1.5441 - val_accuracy: 0.7137 - val_loss: 0.9340
Epoch 4/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5708 - loss: 1.4477 - val_accuracy: 0.7301 - val_loss: 0.8710
Epoch 5/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5796 - loss: 1.4033 - val_accuracy: 0.7298 - val_loss: 0.8557
Epoch 6/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5907 - loss: 1.3714 - val_accuracy: 0.7423 - val_loss: 0.8318
Epoch 7/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5999 - loss: 1.3418 - val_accuracy: 0.7451 - val_loss: 0.8087
Epoch 8/20
1410/1410 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.6077 - loss: 1.3125 -

**Evaluate and compare the models**

In [10]:
# Evaluate the best fine-tuned CNN model
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_val, y_val, verbose=0)
print(f"Best CNN Model - Loss: {cnn_loss:.4f}, Accuracy: {cnn_accuracy:.4f}")

# Evaluate the best fine-tuned MLP model
mlp_loss, mlp_accuracy = mlp_model.evaluate(X_val.reshape(-1, 784), y_val, verbose=0)
print(f"Best MLP Model - Loss: {mlp_loss:.4f}, Accuracy: {mlp_accuracy:.4f}")

# Compare the models
print("\nComparison of Models:")
print(f"CNN Model - Loss: {cnn_loss:.4f}, Accuracy: {cnn_accuracy:.4f}")
print(f"MLP Model - Loss: {mlp_loss:.4f}, Accuracy: {mlp_accuracy:.4f}")


Best CNN Model - Loss: 0.3467, Accuracy: 0.8770
Best MLP Model - Loss: 0.7303, Accuracy: 0.7694

Comparison of Models:
CNN Model - Loss: 0.3467, Accuracy: 0.8770
MLP Model - Loss: 0.7303, Accuracy: 0.7694
